# **PVS-like case study with small-scale sample data**

In this case study, we imagine running the Census Bureau's Person Identification Validation System (PVS) on the 2030 Census Unedited File (CUF). The PVS aims to give people in the input file (the CUF, in this case) unique identifiers that can be used to link them to other administrative records.

We attempt to approximate PVS methods based on publicly available descriptions.
We approximate the (highly confidential) input data to PVS by using simulated data from our pseudopeople package.
See the `generate_simulated_data` notebook for details about this.

## Sources

These papers describe PVS as their main subject:

* Wagner and Layne. The Person Identification Validation System (PVS): Applying the Center for Administrative Records Research and Applications’ (CARRA) Record Linkage Software. 2014. https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-01.pdf ([archived](https://web.archive.org/web/20230216043235/https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-01.pdf))
* Layne, Wagner, and Rothhaas. Estimating Record Linkage False Match Rate for the Person Identification Validation System. 2014. https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-02.pdf ([archived](https://web.archive.org/web/20220121051156/https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-02.pdf))
* Alexander et al. Creating a Longitudinal Data Infrastructure at the Census Bureau. 2015. https://www.census.gov/content/dam/Census/library/working-papers/2015/adrm/2015-alexander.pdf ([archived](https://web.archive.org/web/20170723192315/http://www.census.gov/content/dam/Census/library/working-papers/2015/adrm/2015-alexander.pdf))
* Massey and O'Hara. Person Matching in Historical Files using the Census Bureau’s Person Validation System. 2014. https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-11.pdf ([archived](https://web.archive.org/web/20221018074814/http://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-11.pdf))
* NORC. Assessment of the U.S. Census Bureau’s Person Identification Validation System. 2011. https://www.norc.org/content/dam/norc-org/pdfs/PVS%20Assessment%20Report%20FINAL%20JULY%202011.pdf ([archived](https://web.archive.org/web/20230705005935/https://www.norc.org/content/dam/norc-org/pdfs/PVS%20Assessment%20Report%20FINAL%20JULY%202011.pdf))

These apply PVS to some linking task, and in doing so describe it in some detail:

* Brown et al. Real-Time 2020 Administrative Record Census Simulation. 2023. https://www2.census.gov/programs-surveys/decennial/2020/program-management/evaluate-docs/EAE-2020-admin-records-experiment.pdf ([archived](https://web.archive.org/web/20230521191811/https://www2.census.gov/programs-surveys/decennial/2020/program-management/evaluate-docs/EAE-2020-admin-records-experiment.pdf))
* Massey et al. Linking the 1940 U.S. Census with Modern Data. 2018. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6530596/ ([DOI](https://doi.org/10.1080%2F01615440.2018.1507772))
* Luque and Wagner. Assessing Coverage and Quality of the 2007 Prototype Census Kidlink Database. 2015. https://www.census.gov/content/dam/Census/library/working-papers/2015/adrm/carra-wp-2015-07.pdf ([archived](https://web.archive.org/web/20220808205231/http://www.census.gov/content/dam/Census/library/working-papers/2015/adrm/carra-wp-2015-07.pdf))
* Bond et al. The Nature of the Bias When Studying Only Linkable Person Records: Evidence from the American Community Survey. 2014. https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-08.pdf ([archived](https://web.archive.org/web/20220803083857/http://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-08.pdf))

Finally, this paper is about the address matching process (MAF Match) which acts as part of PVS:

* Brummet. Comparison of Survey, Federal, and Commercial Address Data Quality. 2014. https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-06.pdf ([archived](https://web.archive.org/web/20220121213358/https://www.census.gov/content/dam/Census/library/working-papers/2014/adrm/carra-wp-2014-06.pdf))

All of these papers are referenced throughout this notebook (and other notebooks) by the authors' names, along with additional information like page numbers.
All page numbers represent those of the PDF files themselves, *not* page numbers printed in the documents.

## PVS overview

Brown et al., p. 28:

> The PVS uses probabilistic record linkage (Fellegi and Sunter, 1969) to match data from an
    incoming file (e.g., a survey or administrative record file) to reference files containing data on
    SSN applications from the NUMIDENT enhanced with address data obtained from other federal
    administrative records.

## Setup

In [1]:
import re, copy
import pandas as pd, numpy as np
import jellyfish

In [2]:
# It's not easy to programmatically display direct dependencies separate from transitive
# dependencies.
# This environment was created by:
# $ conda create -n <name> python=3.10
# $ conda activate <name>
# $ pip install pandas numpy matplotlib pseudopeople splink jupyterlab jellyfish
! conda env export | grep -v 'prefix:' | tee conda_environment.yaml

name: pvs_like_case_study
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - bzip2=1.0.8=h7f98852_4
  - ca-certificates=2023.5.7=hbcca054_0
  - ld_impl_linux-64=2.40=h41732ed_0
  - libffi=3.4.2=h7f98852_5
  - libgcc-ng=13.1.0=he5830b7_0
  - libgomp=13.1.0=he5830b7_0
  - libnsl=2.0.0=h7f98852_0
  - libsqlite=3.42.0=h2797004_0
  - libuuid=2.38.1=h0b41bf4_0
  - libzlib=1.2.13=hd590300_5
  - ncurses=6.4=hcb278e6_0
  - openssl=3.1.1=hd590300_1
  - pip=23.1.2=pyhd8ed1ab_0
  - python=3.10.11=he550d4f_0_cpython
  - readline=8.2=h8228510_1
  - setuptools=68.0.0=pyhd8ed1ab_0
  - tk=8.6.12=h27826a3_0
  - wheel=0.40.0=pyhd8ed1ab_0
  - xz=5.2.6=h166bdaf_0
  - pip:
    - altair==5.0.1
    - ansiwrap==0.8.4
    - anyio==3.7.0
    - argon2-cffi==21.3.0
    - argon2-cffi-bindings==21.2.0
    - arrow==1.2.3
    - asttokens==2.2.1
    - async-lru==2.0.2
    - attrs==23.1.0
    - babel==2.12.1
    - backcall==0.2.0
    - beautifulsoup4==4.12.2
    - b

# Load data

See code in `generate_simulated_data` directory for how we generated the files to link

In [3]:
census_2030 = pd.read_parquet('generate_simulated_data/output/census_2030_sample.parquet')
geobase_reference_file = pd.read_parquet('generate_simulated_data/output/geobase_reference_file_sample.parquet')
name_dob_reference_file = pd.read_parquet('generate_simulated_data/output/name_dob_reference_file_sample.parquet')

# Pre-process input file

Wagner and Layne, p. 9:

> The first step of the PVS process is to edit data fields to make them homogenous for
comparisons between incoming and reference files.

In [4]:
# Input file before any processing; the final result will be this with the PIK column added
census_2030_raw_input = census_2030.copy()

## Name parsing and standardizing

Wagner and Layne, p. 9:

> The first edits are parsing and standardizing - parsing separates fields into
component parts, while standardizing guarantees key data elements are consistent (e.g.,
STREET, STR are both converted to ST). Name and address fields are parsed and
standardized as they are key linkage comparators. 

As noted in the data generation notebook, there is no parsing to be done here:
because the Census questionnnaire asks for first name, middle initial, and last name as
separate fields on the form, the name would be already parsed when running PVS on the CUF.

The real PVS name parser generates fields like prefix (e.g. Mr.) and suffix (e.g. Jr.)
but we never have these in names from pseudopeople.

More details about the name parser and standardizer (Wagner and Layne, p. 10):

> The PVS system incorporates a name
standardizer (McGaughey, 1994), which is a C language subroutine called as a function
within SAS. It performs name parsing and includes a nickname lookup table and outputs
name variants (standardized variations of first and last names). For example, Bill
becomes William, Chuck and Charlie becomes Charles, etc. The PVS keeps both the
original name (Bill) and the converted name (William) for matching. PVS also has a fake
name table to blank names such as “Queen of the House” or “Baby Girl.” The name data
are parsed, checked for nicknames, and standardized.

The key thing that wasn't clear to me from this description was what it meant to
"keep both original name and the converted name for matching."
I found this in Massey et al.:

> The Name Search Module also accounts for instances where census records contain a nickname.
> For these records, the preprocessing step of the Name Search module outputs two records for
> these observations, one record for the nickname and one record for the formal name.
> For example, if the input record has the name “Bill Smith,” the formatting program will add
> a formal name “William” to that record. This record will then output to both the B-S cut and to the W-S cut.

From this, I gather that nicknames work similar to alternate names in the reference file:
entire duplicate records are made with each name (nickname and formal).
However, I am still confused by calling this "the preprocessing step **of the Name Search module**."
In Wagner and Layne it does not seem like this is module-specific, and I don't see any
reason that would be desirable, so I have done this multi-record approach across all modules.

In [5]:
# Nickname processing
# Have not yet found a nickname list in PVS docs,
# so we do a minimal version for now -- could use
# another list such as the one in pseudopeople
# These examples all come directly from examples in the descriptions of PVS
nickname_standardizations = {
    "Bill": "William",
    "Chuck": "Charles",
    "Charlie": "Charles",
    "Cathy": "Catherine",
    "Matt": "Matthew",
}
has_nickname = census_2030.first_name.isin(nickname_standardizations.keys())
print(f'{has_nickname.sum()} nicknames in the Census')

# Add extra rows for the normalized names
census_2030 = pd.concat([
    census_2030,
    census_2030[has_nickname].assign(first_name=lambda df: df.first_name.replace(nickname_standardizations))
], ignore_index=True)

# Note: The above will introduce duplicates on record_id, so we redefine
# record_id to be unique (without getting rid of the original, input file record ID)
def add_unique_record_id(df, dataset_name):
    df = df.reset_index().rename(columns={'index': 'record_id'})
    df['record_id'] = f'{dataset_name}_' + df.record_id.astype(str)
    return df

census_2030 = add_unique_record_id(
    census_2030.rename(columns={'record_id': 'record_id_raw_input_file'}),
    "census_2030_preprocessed",
)

13 nicknames in the Census


In [6]:
# This list of fake names comes from the NORC report, p. 100-101
# It is what was used in PVS as of 2011
with open('fake-names.txt') as f:
    fake_names = pd.Series([name.strip().upper() for name in f.readlines()])

assert (fake_names == fake_names.str.upper()).all()
fake_names

0        (CONFIDENTIAL)
1      (NO MIDDLE NAME)
2           A RELUCTANT
3                 ADULT
4               ADULT F
             ...       
766       YOUNGEST DAUG
767       YOUNGEST GIRL
768              YR BOY
769             YR GIRL
770              YR OLD
Length: 771, dtype: object

In [7]:
for col in ["first_name", "last_name"]:
    has_fake_name = census_2030[col].str.upper().isin(fake_names)
    print(f'Found {has_fake_name.sum()} fake names in {col}')
    census_2030[col] = np.where(
        has_fake_name,
        np.nan,
        census_2030[col]
    )

Found 66 fake names in first_name
Found 54 fake names in last_name


## Address parsing and standardizing

Wagner and Layne, p. 11:

> The PVS editing process also incorporates an address parser and standardizer,
written in the C language and called as a function within SAS (U.S. Census Bureau
Geography Division, 1995). It performs parsing of address strings into individual output
fields (see Figure 2), and standardizes the spelling of key components of the address
such as street type. The PVS also incorporates use of a commercial product to update
zip codes, and correct misspellings of address elements.

As noted in the data generation notebook, for now we haven't combined the address parts
into a single string that would need to be parsed.
We plan to add this in a future version of pseudopeople.

For now, we do some simple standardization in each of the address parts (street number,
street name, etc).

In [8]:
def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.split().str.join(' ')
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted from
            # Wagner and Layne p. 9
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', np.nan)
    )

address_cols = ['street_number', 'street_name', 'unit_number', 'city', 'state', 'zipcode']
census_2030[address_cols] = census_2030[address_cols].apply(standardize_address_part)

## MAFMatch

**General information about MAFMatch**

Wagner and Layne, p. 11:

> The PVS provides an additional address enhancement by matching records in the
incoming file to Census Bureau’s Master Address File (MAF) in order to assign a unique
address identifier, the MAF Identifier (MAFID), and other Census geographical codes
(e.g., Census tract and block). The MAFID is used in the PVS for search purposes and as a
linkage key for administrative files. Then, addresses are matched to the Census Bureau’s
Topologically Integrated Geographic Encoding and Referencing Database (TIGER) to
obtain Census geographical codes.

My understanding is that this step is useful for two reasons:
- Potentially adds an alternate version/way of formatting the same address
  (the way it is in the MAF, instead of the way it is in the input file).
  However, instead of using this alternate to match directly, it is boiled down
  to a MAFID, so it is only useful when an input file address and a reference file
  address (different to one another) both match to the same MAF address.
  (Presumably, the whole MAFMatch process described here needs to run on the reference files?)
- Adds Census geographies which could be used in blocking (but are they?)

I don't understand the TIGER part of this.
In particular, p. 12 describes a probabilistic/fuzzy match to TIGER, but I thought TIGER
would contain exactly the same addresses as the MAF.
I also don't understand what geocodes would be present in TIGER that wouldn't also be
present in the MAF.
Maybe these things are more out-of-sync with each other than I understood.

**MAFMatch in this case study**

In the decennial Census, the sampling frame is a subset of the MAF (Brown et al., p. 15, footnote 4).
That is, in the CUF, the address field (which is not supplied by the respondent)
would be the MAF address the questionnaire/NRFU was sent to.

Therefore, I don't believe it makes sense to do MAFMatch for the CUF, because all the addresses
are already the same.
I haven't fully confirmed this, but p. 14 of Wagner and Layne says

> The 2010 Census Unedited File (CUF), had 350 million records and processed through every PVS
module, excluding MAF match and SSN verification

which suggests that (like SSN verification) MAFMatch is not applicable to the CUF.

Given this, we skip MAFMatch here.
Also, if we wanted to add it, we would need to generate something like a MAF from pseudopeople.

## Drop records with insufficient information

In 2011 (NORC, p. 25):

> The initial edit process, described in the Introduction: PVS Background section, removes from consideration
incoming records that have no name data. Therefore, no record that is processed in PVS has blank first and last
names.

In 2014 it appears to be the same/similar, because in Table 2 of Wagner and Layne there is a row "NO SEARCH: Blank Name" (p. 18).

In 2023 (Brown et al., p. 28):

> Records containing sufficient PII to be linkable with some confidence, for example those
containing name and age, are sent through the linkage process.<sup>15</sup>

> <sup>15</sup> Records with names on the PVS invalid name list (e.g., “Mickey Mouse,” “householder,” or “son”) are excluded
from PVS search.

I'd prefer to use the 2023 information, but it is too vague:
"for example" means this is just an approximation, and it doesn't specify
parts of "name."
The footnote also seems to contradict earlier reports, which said fake names were simply
blanked out, which seems preferable.
Since the fake name step comes before this one, a fake first **and** last name would lead
to exclusion here, which is perhaps what the footnote was intending?

Here, we follow the blank-name approach.

In [9]:
census_2030 = census_2030[
    census_2030.first_name.notnull() |
    census_2030.last_name.notnull()
]

# Create derived variables for use in linkage

Here we create variables to be used as matching variables and blocking keys,
when those matching variables/blocking keys are defined in a way that is not already
present in the data at this point.

The variables needed here depend on the modules and passes described below -- see those
sections for more citations.

In [10]:
# We want to compare mailing address with physical address
geobase_reference_file = geobase_reference_file.rename(columns=lambda c: c.replace('mailing_address_', ''))

In [11]:
# PVS uses DOB as separate fields for day, month, and year
def split_dob(df):
    df = df.copy()
    # Have to be floats, because there is missingness, and we want to treat as numeric for assessing similarity
    # Note that as of now, none of our pseudopeople noise types would change the punctuation ("/") in the date, nor would
    # they insert non-numeric characters here.
    # In the future, this might happen, and this simple parsing would need more error handling.
    df[['month_of_birth', 'day_of_birth', 'year_of_birth']] = df.date_of_birth.str.split('/', expand=True).astype(float)
    return df.drop(columns=['date_of_birth'])

census_2030 = split_dob(census_2030)
geobase_reference_file = split_dob(geobase_reference_file)
name_dob_reference_file = split_dob(name_dob_reference_file)

In [12]:
# I don't fully understand the purpose of blocking on the geokey,
# as opposed to just blocking on its constituent columns.
# Maybe it is a way of dealing with missingness in those constituent
# columns (e.g. so an address with no unit number can still be blocked on geokey)?
def add_geokey(df):
    df = df.copy()
    df['geokey'] = (
        df.street_number + ' ' +
        df.street_name + ' ' +
        df.unit_number.fillna('') + ' ' +
        df.city + ' ' +
        df.state.astype(str) + ' ' +
        df.zipcode
    )
    # Normalize the whitespace -- necessary if the unit number was null
    df['geokey'] = (
        df.geokey.str.split().str.join(' ')
    )
    return df

geobase_reference_file = add_geokey(geobase_reference_file)
census_2030 = add_geokey(census_2030)

In [13]:
# Layne, Wagner, and Rothhaas p. 26: the name matching variables are
# First 15 characters First Name, First 15 characters Middle Name, First 12 characters Last Name
# Additionally, there are blocking columns for all of 1-3 initial characters of First/Last.
# We don't have a full middle name in pseudopeople (nor would that be present in a real CUF)
# so we have to stick to the first initial for middle.
def add_truncated_name_cols(df):
    df = df.copy()
    df['first_name_15'] = df.first_name.str[:15]
    df['last_name_12'] = df.last_name.str[:12]

    for num_chars in [1, 2, 3]:
        df[f'first_name_{num_chars}'] = df.first_name.str[:num_chars]
        df[f'last_name_{num_chars}'] = df.last_name.str[:num_chars]

    return df

census_2030 = add_truncated_name_cols(census_2030)
geobase_reference_file = add_truncated_name_cols(geobase_reference_file)
name_dob_reference_file = add_truncated_name_cols(name_dob_reference_file)

In [14]:
# Layne, Wagner, and Rothhaas p. 26: phonetics are used in blocking (not matching)
# - Soundex for Street Name
# - NYSIIS code for First Name
# - NYSIIS code for Last Name
# - Reverse Soundex for First Name
# - Reverse Soundex for Last Name

def add_name_phonetics(df):
    df = df.copy()

    for col in ['first_name', 'last_name']:
        df[f'{col}_nysiis'] = df[col].dropna().apply(jellyfish.nysiis)
        df[f'{col}_reverse_soundex'] = df[col].dropna().str[::-1].apply(jellyfish.soundex)

    return df

def add_address_phonetics(df):
    df = df.copy()
    df['street_name_soundex'] = df.street_name.dropna().apply(jellyfish.soundex)
    return df

census_2030 = add_name_phonetics(census_2030)
census_2030 = add_address_phonetics(census_2030)

geobase_reference_file = add_address_phonetics(geobase_reference_file)

name_dob_reference_file = add_name_phonetics(name_dob_reference_file)

In [15]:
# Columns used to "cut the database": ZIP3 and a grouping of first and last initial
def add_zip3(df):
    return df.assign(zip3=lambda x: x.zipcode.str[:3])

def add_first_last_initial_categories(df):
    # Page 20 of the NORC report: "Name-cuts are defined by combinations of the first characters of the first and last names. The twenty letter groupings
    # for the first character are: A-or-blank, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, and U-Z."
    initial_cut = lambda x: x.fillna('A').str[0].replace('A', 'A-or-blank').replace(['U', 'V', 'W', 'X', 'Y', 'Z'], 'U-Z')
    return df.assign(first_initial_cut=lambda x: initial_cut(x.first_name), last_initial_cut=lambda x: initial_cut(x.last_name))

In [16]:
census_2030 = add_zip3(census_2030)
census_2030 = add_first_last_initial_categories(census_2030)

geobase_reference_file = add_zip3(geobase_reference_file)

name_dob_reference_file = add_first_last_initial_categories(name_dob_reference_file)

# Data, ready to link

In [17]:
census_2030

,record_id,record_id_raw_input_file,first_name,middle_initial,last_name,age,street_number,street_name,unit_number,city,...,first_name_3,last_name_3,first_name_nysiis,first_name_reverse_soundex,last_name_nysiis,last_name_reverse_soundex,street_name_soundex,zip3,first_initial_cut,last_initial_cut
0,census_2030_preprocessed_0,census_2030_0,John,E,Mcueever,86,147-153,BROWNING AVE,None,ANYTOWN,...,Joh,Mcu,JAN,N200,MCAAFAR,R125,B655,000,J,M
1,census_2030_preprocessed_1,census_2030_1,Sharon,T,Schmidt,69,109,STQLLION SR,None,ANYTOWN,...,Sha,Sch,SARAN,N620,SNAD,T520,S324,000,S,S
2,census_2030_preprocessed_2,census_2030_2,Gail,K,Durand,77,2115,CANNON DR,None,ANYTOWN,...,Gai,Dur,GAL,L200,DARAD,D563,C553,000,G,D
3,census_2030_preprocessed_3,census_2030_3,John,J,Williams,81,146,DELAWARE AV,None,ANYTOWN,...,Joh,Wil,JAN,N200,WALAN,S540,D461,000,J,U-Z
4,census_2030_preprocessed_4,census_2030_4,NaN,L,Wukliamz,81,146,DELAWARE AV,None,ANYTOWN,...,NaN,Wuk,NaN,NaN,WACLAN,Z542,D461,000,A-or-blank,U-Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11061,census_2030_preprocessed_11061,census_2030_7991,Charles,B,Moser,29,289,COLUMBUS DR,None,ANYTOWN,...,Cha,Mos,CARL,S462,MASAR,R250,C451,000,C,M
11062,census_2030_preprocessed_11062,census_2030_8622,Catherine,C,Blunt,50,11467,LENORE,None,ANYTOWN,...,Cat,Blu,CATARAN,E563,BLAD,T541,L560,000,C,B
11063,census_2030_preprocessed_11063,census_2030_9128,Charles,Q,Scott,16,2139,LABRADOR RUN N,None,ANYTOWN,...,Cha,Sco,CARL,S462,SCAT,T200,L163,000,C,S
11064,census_2030_preprocessed_11064,census_2030_10237,Matthew,C,Pineda,58,5,SW 115TH TER,None,ANYTOWN,...,Mat,Pin,MATAE,W350,PANAD,A351,S336,000,M,P


In [18]:
geobase_reference_file

,record_id,ssn,first_name,middle_initial,last_name,street_number,street_name,unit_number,po_box,city,...,first_name_15,last_name_12,first_name_1,last_name_1,first_name_2,last_name_2,first_name_3,last_name_3,street_name_soundex,zip3
0,geobase_reference_file_0,001-02-4588,Isabella,G,Windom,685,EMERSON ST,None,None,ANYTOWN,...,Isabella,Windom,I,W,Is,Wi,Isa,Win,E562,000
1,geobase_reference_file_1,001-15-8330,Gerald,J,Beckham,5010,SOUTH DOCTOR MARTIN LUTHER KING JR DR,None,None,ANYTOWN,...,Gerald,Beckham,G,B,Ge,Be,Ger,Bec,S332,000
2,geobase_reference_file_2,001-15-8330,Gerald,J,Beckham,5010,SOUTH DOCTOR MARTIN LUTHER KING JR DR,None,None,ANYTOWN,...,Gerald,Beckham,G,B,Ge,Be,Ger,Bec,S332,000
3,geobase_reference_file_3,001-15-8330,Gerald,J,Beckham,5010,SOUTH DOCTOR NARTIN LURHER KING JR DR,None,None,ANYTOWN,...,Gerald,Beckham,G,B,Ge,Be,Ger,Bec,S332,000
4,geobase_reference_file_4,001-16-0077,Jerald,J,Alvarez,None,None,None,None,None,...,Jerald,Alvarez,J,A,Je,Al,Jer,Alv,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34207,geobase_reference_file_34207,996-41-6227,Dorothy,J,Faria,1702,MEISNER RD,None,None,ANYTOWN,...,Dorothy,Faria,D,F,Do,Fa,Dor,Far,M256,000
34208,geobase_reference_file_34208,997-63-6760,Sofia,A,Garc,10900,S K ST,None,None,ANYTOWN,...,Sofia,Garc,S,G,So,Ga,Sof,Gar,S223,000
34209,geobase_reference_file_34209,997-78-4873,Penny,L,Mccauley,8370,CHERVIL CT,None,None,ANYTOWN,...,Penny,Mccauley,P,M,Pe,Mc,Pen,Mcc,C614,000
34210,geobase_reference_file_34210,997-91-2268,Jeremy,None,Mast,3232,MAPLE GROVE LN,None,None,ANYTOWN,...,Jeremy,Mast,J,M,Je,Ma,Jer,Mas,M142,000


In [19]:
name_dob_reference_file

,record_id,ssn,first_name,middle_initial,last_name,pik,month_of_birth,day_of_birth,year_of_birth,first_name_15,...,first_name_2,last_name_2,first_name_3,last_name_3,first_name_nysiis,first_name_reverse_soundex,last_name_nysiis,last_name_reverse_soundex,first_initial_cut,last_initial_cut
0,name_dob_reference_file_0,001-02-4588,Isabella,G,Windom,452,8.0,8.0,2008.0,Isabella,...,Is,Wi,Isa,Win,ISABAL,A412,WANDAN,M350,I,U-Z
1,name_dob_reference_file_1,001-15-8330,Gerald,J,Beckham,5866,5.0,4.0,1976.0,Gerald,...,Ge,Be,Ger,Bec,GARALD,D462,BACKAN,M210,G,B
2,name_dob_reference_file_2,001-16-0077,Jerald,J,Alvarez,4761,2.0,7.0,1970.0,Jerald,...,Je,Al,Jer,Alv,JARALD,D462,ALVAR,Z614,J,A-or-blank
3,name_dob_reference_file_3,001-17-9511,Teresa,A,Togni,4172,11.0,20.0,1966.0,Teresa,...,Te,To,Ter,Tog,TARAS,A263,TAGN,I523,T,T
4,name_dob_reference_file_4,001-25-8258,Bethany,G,Tenorio,17384,6.0,29.0,2026.0,Bethany,...,Be,Te,Bet,Ten,BATANY,Y531,TANAR,O653,B,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19527,name_dob_reference_file_19527,996-41-6227,Dorothy,J,Faria,91098,NaN,NaN,NaN,Dorothy,...,Do,Fa,Dor,Far,DARATY,Y363,FAR,A610,D,F
19528,name_dob_reference_file_19528,997-63-6760,Sofia,A,Garc,91099,NaN,NaN,NaN,Sofia,...,So,Ga,Sof,Gar,SAF,A120,GARC,C620,S,G
19529,name_dob_reference_file_19529,997-78-4873,Penny,L,Mccauley,91100,NaN,NaN,NaN,Penny,...,Pe,Mc,Pen,Mcc,PANY,Y510,MCALY,Y425,P,M
19530,name_dob_reference_file_19530,997-91-2268,Jeremy,None,Mast,91101,NaN,NaN,NaN,Jeremy,...,Je,Ma,Jer,Mas,JARANY,Y562,MAST,T250,J,M


# Emulate Multi-Match with splink

Wagner and Layne, p. 8:

> The PVS employs its probabilistic record linkage software, Multi-Match (Wagner
2012), as an integral part of the PVS.

Wagner and Layne, p. 12:

> PVS uses the same Multi-Match engine for each probabilistic search type. For each
search module the analyst defines a parameter file, which is passed to Multi-Match. The
parameter file includes threshold value(s) for the number of passes, blocking keys, and
within each pass, the match variables, match comparison type, and matching weights...
>
> Records must first match exactly on the blocking keys before any comparisons
between the match variables are attempted. Each match variable is given an
m and
u
probability, which is translated by MultiMatch as agreement and disagreement weights.
The sum of all match variable comparison weights for a record pair is the composite
weight. All record pairs with a composite weight greater than or equal to the threshold
set in the parameter file are linked, and the records from the incoming file for these
linked cases are excluded from all remaining passes. All Numident records are always
available for linking in every pass. Any record missing data for any of the blocking fields
for a pass skips that pass and moves to the next pass.

[splink](https://github.com/moj-analytical-services/splink) is similar to Multi-Match
(both are based on the Fellegi-Sunter approach to record linkage).
It also implements:
- Exact blocking on specified keys
- Determining overall match probability based on conditional independence of individual field comparisons
  (it is equivalent to multiply the probability ratios, or sum the logarithmic "weights" as Multi-Match describes it),
  which each have m and u probabilities
- Setting a match threshold

However, splink does not include some additional logic built into Multi-Match,
specifically the ability to run multiple "passes," removing linked records from
subsequent passes.
We have to implement that ourselves, calling splink again in each pass.

## Estimate parameters (lambda, m, u) once for all modules

In Multi-Match, there is no lambda (prior probability of a link, before any comparisons are observed).
This is because Multi-Match works entirely in weight space, and sets different weight thresholds
in different modules/passes instead of changing the prior probability.

In Multi-Match parameters are not directly estimated from the data.
They are primarily set manually by analysts, with a different set of parameter files
maintained for each type of input file (e.g. survey, administrative).

All parameters except lambda are column-level parameters.
We estimate the column-level parameters using the GeoBase Reference File,
since it has all columns that are used for matching with any reference file.

We estimate lambda (manually, see below) separately by reference file,
since the number of records is different between the reference files but
the size of the underlying population (people we are trying to match) is constant.

### lambda (prior probability of a match)

Splink has a built-in method (estimate_probability_two_random_records_match)
for estimating this, but it did not seem to give me reasonable estimates.

We just make an informed guess here based on how much overlap we expect
in the files, how much unintentional duplication we expect in the files (e.g. someone
being enumerated twice), how much *intentional* duplication we have in the
files (e.g. a record for each nickname variant), and some simplifying assumptions.

Our assumptions:
- 5% of the enumerations in the CUF are unintentional duplicates
- 0.5% of the PIKs in the reference file are unintentional duplicates
  (that same person is also represented with a different PIK)
- 90% of the people in the CUF are represented in the reference files
- Being represented in both files is independent of being unintentionally
  or intentionally duplicated
- Being intentionally duplicated in one file is independent of being intentionally
  duplicated in the other (likely not true since having a name with lots of variants
  would cause intentional duplicates in both)

We do this first (before m and u probabilities) because having lambda estimated is
useful to the EM algorithm for estimating m and u.

In [20]:
def estimate_number_true_matches(input_file, reference_file):
    people_represented_in_input_file = (
        input_file.record_id_raw_input_file.nunique() * 0.95
    )
    people_represented_in_reference_file = (
        reference_file.pik.nunique() * 0.995
    )
    people_represented_in_both = people_represented_in_input_file * 0.9

    # Assuming independence conditions as noted above, the number of true
    # matches that should be found for *each* true person is the expected number
    # of records in one file times the expected number of records in the other
    input_file_records_per_person = people_represented_in_input_file / len(input_file)
    reference_file_records_per_person = people_represented_in_reference_file / len(reference_file)
    record_matches_per_person = input_file_records_per_person * reference_file_records_per_person

    return people_represented_in_both * record_matches_per_person

def probability_two_random_records_match(input_file, reference_file):
    cartesian_product = len(input_file) * len(reference_file)
    return (
        estimate_number_true_matches(input_file, reference_file) /
        cartesian_product
    )

probability_two_random_records_match(census_2030, geobase_reference_file)

1.3024390660875874e-05

### m and u probabilities

In [21]:
common_cols = [c for c in census_2030.columns if c in geobase_reference_file.columns or c in name_dob_reference_file.columns]
common_cols

['record_id',
 'first_name',
 'middle_initial',
 'last_name',
 'street_number',
 'street_name',
 'unit_number',
 'city',
 'state',
 'zipcode',
 'month_of_birth',
 'day_of_birth',
 'year_of_birth',
 'geokey',
 'first_name_15',
 'last_name_12',
 'first_name_1',
 'last_name_1',
 'first_name_2',
 'last_name_2',
 'first_name_3',
 'last_name_3',
 'first_name_nysiis',
 'first_name_reverse_soundex',
 'last_name_nysiis',
 'last_name_reverse_soundex',
 'street_name_soundex',
 'zip3',
 'first_initial_cut',
 'last_initial_cut']

In [22]:
def prep_table_for_splink(df, dataset_name, columns):
    return (
        df[[c for c in df.columns if c in columns]]
            .assign(dataset_name=dataset_name)
    )

tables_for_splink = [
    prep_table_for_splink(geobase_reference_file, "geobase_reference_file", common_cols),
    prep_table_for_splink(census_2030, "census_2030", common_cols)
]

In [23]:
[len(t) for t in tables_for_splink]

[34212, 11065]

#### Define comparison variables and levels

**Variables**

The most recent report, from 2023 (Brown et al., p. 29), says:

> [In] GeoSearch... The typical matching variables are name, DOB, sex, and
  various address fields...
>
> NameSearch... uses only name and DOB fields...
> 
> DOBSearch... compares name, sex, and DOB...
> 
> the Household Composition search module... attempts to find a match... based on name and DOB information

So, the total list: name, DOB, sex, and "various address fields."
These address fields are listed in the PVS report (p. 34) which is admittedly from 2011:

> variables used directly in matching the input file with the reference files [include]...
> street name, street name prefix and suffix, house number, rural route and box, and ZIP code

**Comparisons**

Massey et al. footnote 2: 
> The PVS string comparator was developed by Winkler (1995) and measures the distance
  between two strings on a scale from 0 to 900, where a distance score of 0 is given if
  there is no similarity between two text strings and a score of 900 is given for an exact match.
  The cutoff value for the string distance is set to 750 in the Name Search module.

Massey and O'Hara, p. 6 footnote 7:
> For the comparison of text strings, a prorated value
  between the chosen agreement score and chosen disagreement score is given depending on the Jaro-Winkler
  distance between the string in the input file and reference file.

Massey et al.:
> For numeric variables, such as year of birth, a maximum acceptable difference
> between the variable value in the input and reference record is set. This also
> allows for creation of an interval, or band, around year of birth to permit
> inexact matches. Within this band, prorated agreement and disagreement weights
> are assigned depending on the similarity of year of birth.

The "maximum acceptable distance" implies that the m probability beyond that distance is
zero; such pairs should never be linked.

Note that this continuous "prorated value," as opposed to categorizing comparison levels,
is not possible in splink and goes outside the traditional F-S framing of m and u probabilities!
We omit it for now.

In [24]:
from splink.duckdb.linker import DuckDBLinker
from splink.duckdb.comparison_library import (
    exact_match,
    jaro_winkler_at_thresholds,
)
import splink.duckdb.comparison_level_library as cll

def numeric_column_comparison(col_name, human_name, maximum_inexact_match_difference):
    return {
        "output_column_name": col_name,
        "comparison_description": human_name,
        "comparison_levels": [
            cll.null_level(col_name),
            cll.exact_match_level(col_name),
            {
                "sql_condition": f"abs({col_name}_l - {col_name}_r) <= {maximum_inexact_match_difference}",
                "label_for_charts": "Inexact match",
            },
            cll.else_level(),
        ],
    }

numeric_columns = ["day_of_birth", "month_of_birth", "year_of_birth"]

settings = {
    "link_type": "link_only",
    "comparisons": [
        jaro_winkler_at_thresholds("first_name_15", 750 / 900),
        jaro_winkler_at_thresholds("last_name_12", 750 / 900),
        exact_match("middle_initial"),
        numeric_column_comparison("day_of_birth", "Day of birth", maximum_inexact_match_difference=5),
        numeric_column_comparison("month_of_birth", "Month of birth", maximum_inexact_match_difference=3),
        numeric_column_comparison("year_of_birth", "Year of birth", maximum_inexact_match_difference=5),
        # Using same cutoffs as for names, in the absence of a better description of
        # how these are compared
        jaro_winkler_at_thresholds("street_number", 750 / 900),
        jaro_winkler_at_thresholds("street_name", 750 / 900),
        jaro_winkler_at_thresholds("unit_number", 750 / 900),
        jaro_winkler_at_thresholds("zipcode", 750 / 900),
    ],
    "probability_two_random_records_match": probability_two_random_records_match(census_2030, geobase_reference_file),
    "unique_id_column_name": "record_id",
    # Not sure exactly what this does, but it is necessary for some of the fancier graphs below
    "retain_intermediate_calculation_columns": True,
}

linker = DuckDBLinker(
    tables_for_splink,
    settings,
    input_table_aliases=["reference_file", "census_2030"]
)

#### Estimate u probabilities

This method seems to work well:
- For almost all of the columns, an exact or even inexact match is relatively rare
- Exact matches are common on ZIP code (makes sense, given our sample data is all in one ZIP)
- Inexact matches are more common where you would expect (day and month of birth have highly constrained values)

I have tested that this estimation is reproducible when run multiple times (with the same seed).

In [25]:
%%time

linker.estimate_u_using_random_sampling(max_pairs=1e7, seed=1234)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name_15 (no m values are trained).
    - last_name_12 (no m values are trained).
    - middle_initial (no m values are trained).
    - day_of_birth (no m values are trained).
    - month_of_birth (no m values are trained).
    - year_of_birth (no m values are trained).
    - street_number (no m values are trained).
    - street_name (no m values are trained).
    - unit_number (no m values are trained).
    - zipcode (no m values are trained).


CPU times: user 34.1 s, sys: 197 ms, total: 34.3 s
Wall time: 4.24 s


In [26]:
# Ignore the green bars on the left, these are the m probabilities that haven't been estimated yet
linker.m_u_parameters_chart()

alt.HConcatChart(...)

#### Estimate m probabilities

The EM algorithm implemented in Splink can be used to estimate all the parameters at once.
However, I've found this to be extremely unstable, and the lambda and u estimates I've made
above much better than what happens when I allow EM to mess with them.

These EM session blocking rules are the result of **lots** of trial and error.
I consistently had problems with the EM algorithm deciding that first names
were completely wrong ("All other comparisons") almost as often as they were
right.
I can think of two reasons this might be happening:
- The EM algorithm is just looking for two coherent clusters, and the assumption is that these
  clusters correspond to matches and non-matches.
  But in real life, and in our simulated data, matches and non-matches are not homogenous.
  In particular, people who are in the same family live together and tend to share a last name,
  and this could have been the cluster EM was finding instead of finding the exact same person.
- Conditional independence is grossly violated by some of our columns, especially the address parts
  and the DOB parts. This could be causing some pathological behavior.

Using the EM approach below, I have at least obtained reasonable-looking m and u probabilities.

In [27]:
blocking_rule_for_training = "l.first_name_15 = r.first_name_15"
em_session_1 = linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule_for_training,
    # Fix lambda; u is fixed by default
    fix_probability_two_random_records_match=True,
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.first_name_15 = r.first_name_15

Parameter estimates will be made for the following comparison(s):
    - last_name_12
    - middle_initial
    - day_of_birth
    - month_of_birth
    - year_of_birth
    - street_number
    - street_name
    - unit_number
    - zipcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name_15

Iteration 1: Largest change in params was -0.311 in the m_probability of street_number, level `Exact match`
Iteration 2: Largest change in params was 0.0212 in the m_probability of street_name, level `All other comparisons`
Iteration 3: Largest change in params was 0.00073 in the m_probability of street_name, level `All other comparisons`
Iteration 4: Largest change in params was -9.37e-05 in the m_probability of year_of_birth, level `Exact match`

EM converged after 4 iterations

Your mo

In [28]:
em_session_1.m_u_values_interactive_history_chart()

alt.HConcatChart(...)

In [29]:
em_session_1.match_weights_interactive_history_chart()

alt.VConcatChart(...)

In [30]:
blocking_rule_for_training = "l.middle_initial = r.middle_initial and l.last_name_12 = r.last_name_12"
em_session_2 = linker.estimate_parameters_using_expectation_maximisation(
    blocking_rule_for_training,
    # Fix lambda; u is fixed by default
    fix_probability_two_random_records_match=True,
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.middle_initial = r.middle_initial and l.last_name_12 = r.last_name_12

Parameter estimates will be made for the following comparison(s):
    - first_name_15
    - day_of_birth
    - month_of_birth
    - year_of_birth
    - street_number
    - street_name
    - unit_number
    - zipcode

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - last_name_12
    - middle_initial

Iteration 1: Largest change in params was -0.0638 in the m_probability of first_name_15, level `Exact match`
Iteration 2: Largest change in params was -0.00217 in the m_probability of first_name_15, level `Exact match`
Iteration 3: Largest change in params was -0.000479 in the m_probability of year_of_birth, level `Exact match`
Iteration 4: Largest change in params was -0.000118 in the m_probability of year_of_birth, level `Exact match`
Iteration 5: L

In [31]:
em_session_2.m_u_values_interactive_history_chart()

alt.HConcatChart(...)

In [32]:
linker.match_weights_chart()

alt.VConcatChart(...)

In [33]:
linker.m_u_parameters_chart()

alt.HConcatChart(...)

In [34]:
linker.parameter_estimate_comparisons_chart()

alt.Chart(...)

In [35]:
splink_settings = linker._settings_obj.as_dict()

In [36]:
# Save these variables; this means that if you restart the kernel, you don't need to run this first part of the notebook again.
%store splink_settings

Stored 'splink_settings' (dict)


## Implement matching passes

In [37]:
from dataclasses import dataclass

# Calculate this once to save time -- mapping from record_id to record_id_raw_input_file
# There can be multiple records (with different record_id) for the same input file record
# (record_id_raw_input_file) because of the handling of nicknames by creating extra records.
record_id_raw_input_file_by_record_id = census_2030.set_index('record_id').record_id_raw_input_file

all_piks = pd.concat([
    name_dob_reference_file[["record_id", "pik"]],
    geobase_reference_file[["record_id", "pik"]]
], ignore_index=True).set_index("record_id").pik

dates_of_death = (
    pd.read_parquet('generate_simulated_data/output/census_numident_sample.parquet')
        .set_index('pik')
        .date_of_death
        .pipe(lambda s: pd.to_datetime(s, format='%Y%m%d', errors='coerce'))
)

class PVSCascade:
    def __init__(self):
        # This dataframe will accumulate the PIKs to attach to the input file
        self.confirmed_piks = pd.DataFrame(columns=["record_id_raw_input_file", "pik"])
        self.current_module = None

    def start_module(self, *args, **kwargs):
        assert self.current_module is None or self.current_module.confirmed
        self.current_module = PVSModule(*args, **kwargs)

    def run_matching_pass(self, *args, **kwargs):
        self.current_module.run_matching_pass(*args, already_confirmed_piks=self.confirmed_piks, **kwargs)

    def confirm_piks(self, *args, **kwargs):
        # Make sure we are not about to confirm PIKs for any of the same files we have
        # already PIKed
        assert (
            set(self.current_module.provisional_links.record_id_raw_input_file) &
            set(self.confirmed_piks.record_id_raw_input_file)
        ) == set()

        newly_confirmed_piks = self.current_module.confirm_piks_from_provisional_links()

        self.confirmed_piks = pd.concat([
            self.confirmed_piks,
            newly_confirmed_piks,
        ], ignore_index=True)

        return self.confirmed_piks

@dataclass
class PVSModule:
    name: str
    reference_file: pd.DataFrame
    reference_file_name: str
    cut_columns: list[str]
    matching_columns: list[str]

    def __post_init__(self):
        self.provisional_links = pd.DataFrame(columns=["record_id_census_2030"])
        self.confirmed = False

    def run_matching_pass(
        self,
        pass_name,
        blocking_columns,
        probability_threshold=0.99,
        input_data_transformation=lambda x: x,
        already_confirmed_piks=pd.DataFrame(columns=["record_id_raw_input_file"]),
    ):
        assert self.confirmed == False

        print(f"Running {pass_name} of {self.name}")

        columns_needed = ["record_id"] + self.cut_columns + blocking_columns + self.matching_columns
        tables_for_splink = [
            prep_table_for_splink(self.reference_file, self.reference_file_name, columns_needed),
            prep_table_for_splink(
                census_2030[
                    # Only look for matches among records that have not received a confirmed PIK
                    ~census_2030.record_id_raw_input_file.isin(already_confirmed_piks.record_id_raw_input_file) &
                    # Only look for matches among records that have not received a provisional link
                    # NOTE: "records" here does not mean input file records -- a nickname record having
                    # a provisional link does not prevent a canonical name record from the same input record
                    # from continuing to match
                    ~census_2030.record_id.isin(self.provisional_links.record_id_census_2030)
                ].pipe(input_data_transformation),
                "census_2030",
                columns_needed,
            )
        ]
        print(f"Files to link are {len(tables_for_splink[0]):,.0f} and {len(tables_for_splink[1]):,.0f} records")

        blocking_rule_parts = [f"l.{col} = r.{col}" for col in self.cut_columns + blocking_columns]
        blocking_rule = " and ".join(blocking_rule_parts)

        # We base our Splink linker for this pass on the one we trained above,
        # but limiting it to the relevant column comparisons and updating the pass-specific
        # settings
        pass_splink_settings = copy.deepcopy(splink_settings)
        pass_splink_settings["comparisons"] = [
            c for c in pass_splink_settings["comparisons"] if c["output_column_name"] in self.matching_columns
        ]
        pass_splink_settings["probability_two_random_records_match"] = (
            probability_two_random_records_match(census_2030, self.reference_file)
        )
        pass_splink_settings["blocking_rules_to_generate_predictions"] = [blocking_rule]
    
        linker = DuckDBLinker(
            tables_for_splink,
            pass_splink_settings,
            # Must match order of tables_for_splink
            input_table_aliases=["reference_file", "census_2030"]
        )
    
        num_comparisons = linker.count_num_comparisons_from_blocking_rule(blocking_rule)
        print(f"Number of pairs that will be compared: {num_comparisons:,.0f}")
    
        # https://moj-analytical-services.github.io/splink/demos/06_Visualising_predictions.html#comparison-viewer-dashboard
        # We also include some pairs below the threshold, for additional context.
        pairs_worth_inspecting = linker.predict(threshold_match_probability=probability_threshold - 0.2)
    
        dashboard_file_name = f"splink_temp/{self.name.replace(' ', '_')}__{pass_name.replace(' ', '_')}.html"
        linker.comparison_viewer_dashboard(pairs_worth_inspecting, dashboard_file_name, overwrite=True)
        from IPython.display import IFrame, display
        display(IFrame(
            src=f"./{dashboard_file_name}", width="100%", height=1200
        ))
    
        new_provisional_links = (
            pairs_worth_inspecting
                .as_pandas_dataframe()
                .pipe(lambda df: df[df.match_probability >= probability_threshold])
        )
        if len(new_provisional_links) > 0:
            new_provisional_links = label_pairs_with_dataset(new_provisional_links)
            new_provisional_links["record_id_raw_input_file"] = (
                new_provisional_links.record_id_census_2030.map(record_id_raw_input_file_by_record_id)
            )
        
            self.provisional_links = pd.concat([
                self.provisional_links,
                new_provisional_links.assign(module_name=self.name, pass_name=pass_name)
            ], ignore_index=True)
    
        still_eligible = (
            (~census_2030.record_id_raw_input_file.isin(already_confirmed_piks.record_id_raw_input_file)) &
            (~census_2030.record_id.isin(self.provisional_links.record_id_census_2030))
        )
        print(f'Matched {len(new_provisional_links)} records; {still_eligible.mean():.2%} still eligible to match')

    def confirm_piks_from_provisional_links(self):
        assert not self.confirmed

        provisional_links = self.provisional_links
        provisional_links["pik"] = provisional_links.record_id_reference_file.map(all_piks)

        # "After the initial set of links is created in GeoSearch, a post-search program is run to determine
        # which of the links are retained. A series of checks are performed: First the date of death
        # information from the Numident is checked and links to deceased persons are dropped. Next a
        # check is made for more than one SSN assigned to a source record. If more than one SSN is
        # assigned, the best link is selected based on match weights. If no best SSN is determined, all SSNs
        # assigned in the GeoSearch module are dropped and the input record cascades to the next
        # module. A similar post-search program is run at the end of all search modules."
        # - Layne et al. p. 5

        # Drop links to deceased people
        # NOTE: On p. 38 of Brown et al. (2023) it discusses at length the number of PVS matches to deceased
        # people, which should not be possible based on this process.
        # Even though this is more recent, I can't think of a reason why this check would have
        # been *removed* from PVS -- can we chalk this up to something experimental they were doing for
        # the AR Census in the 2023 report?
        link_dates_of_death = provisional_links["pik"].map(dates_of_death)
        # Census day 2030
        deceased_links = link_dates_of_death <= pd.to_datetime("2030-04-01")
        print(f'{deceased_links.sum()} input records linked to deceased people, dropping links')
        provisional_links = provisional_links[~deceased_links]

        # Check for multiple linkage to a single input file record
        max_probability = provisional_links.groupby("record_id_raw_input_file").match_probability.max()
        piks_per_input_file = (
            provisional_links.groupby("record_id_raw_input_file")
                .apply(lambda df: df[df.match_probability == max_probability[df.name]].pik.nunique())
        )

        multiple_piks = piks_per_input_file[piks_per_input_file > 1].index
        print(f'{len(multiple_piks)} input records linked to multiple PIKs, dropping links')
        provisional_links = (
            provisional_links[~provisional_links.record_id_raw_input_file.isin(multiple_piks)]
                .sort_values("match_probability")
                .groupby("record_id_raw_input_file")
                .last()
                .reset_index()
        )

        assert (provisional_links.groupby("record_id_raw_input_file").pik.nunique() == 1).all()

        self.confirmed = True
        self.provisional_links = None
        
        return (
            provisional_links[[
                "record_id_raw_input_file",
                "record_id_census_2030",
                "record_id_reference_file",
                "pik",
                "module_name",
                "pass_name",
                "match_probability",
            ]]
        )

def label_pairs_with_dataset(pairs):
    # Name the columns according to the datasets, not "r" (right) and "l" (left)
    for suffix in ["r", "l"]:
        pairs = (
            pairs.groupby(f'source_dataset_{suffix}')
                .apply(lambda df_g: replace_suffix_with_source_dataset(df_g, suffix, df_g.name))
        )

    return pairs

def replace_suffix_with_source_dataset(df, suffix, source_dataset):
    return df.rename(columns=lambda c: re.sub(f'_{suffix}$', f'_{source_dataset}', c))

In [38]:
pvs_cascade = PVSCascade()

# Verification Module

Wagner and Layne, p.14:

> If the input file has a SSN data field, it first goes through the verification process.

This module of PVS only runs on input records with an SSN.
No records in the CUF have an SSN (SSN is not collected on the decennial)
so this module is skipped.

# GeoSearch

Brown et al. p. 29:

> the GeoSearch module blocks records at
different levels of geography, starting with the housing unit level, then broadening geography
up to the three-digit ZIP Code level. The typical matching variables are name, DOB, sex, and
various address fields.

Wagner and Layne p. 14:

> The typical GeoSearch blocking strategy starts with blocking records at the
household level, then broadens the geography for each successive pass and ends at
blocking by the first three digits of the zip code. The typical match variables are first,
middle, and last names; generational suffix; date of birth; gender and various address
fields.
> 
> The data for the GeoSearch module are split into 1,000 cuts based on the first three
digits of the zip code (zip3) for record. The GeoSearch program works on one zip3 cut
at a time, with shell scripts submitting multiple streams of cuts to the system. This
allows for parallel processing and restart capability.

Layne, Wagner, and Rothhaas list an exact series of passes with blocking and matching variables
for each (Appendix A on p. 25).
This was the series of passes they used in running PVS on the Medicare Enrollment
Database, the Indian Health Service Patient Registration File, and on two commercial
data sources.
All of these data sources are rather different than a CUF, and this paper is from 2014,
but it is the only place I know of where an entire set of passes is described,
so I have used the exact same passes here.

In [39]:
pvs_cascade.start_module(
    name="geosearch",
    reference_file=geobase_reference_file,
    reference_file_name="geobase_reference_file",
    cut_columns=["zip3"],
    matching_columns=[
        "first_name_15",
        "last_name_12",
        "middle_initial",
        "day_of_birth",
        "month_of_birth",
        "year_of_birth",
        "street_number",
        "street_name",
        "unit_number",
        "zipcode",
    ],
)

## Pass 1: Block on geokey (entire address)

In [40]:
pvs_cascade.run_matching_pass(
    pass_name="geokey",
    blocking_columns=["geokey"],
)

Running geokey of geosearch
Files to link are 34,212 and 11,065 records
Number of pairs that will be compared: 623,668


Matched 6141 records; 45.37% still eligible to match


## Pass 2: Block on geokey (entire address), switching first and last names

In [41]:
def switch_first_and_last_names(df):
    return (
        df.rename(columns={"first_name": "last_name", "last_name": "first_name"})
            # Re-calculate the truncated versions of first and last.
            # NOTE: It is not necessary to re-calculate the phonetic versions, because
            # those are never used in any pass that has a name switch.
            .pipe(add_truncated_name_cols)
    )

In [42]:
# We don't actually have any swapping of first and last names in pseudopeople
pvs_cascade.run_matching_pass(
    pass_name="geokey name switch",
    blocking_columns=["geokey"],
    input_data_transformation=switch_first_and_last_names,
)

Running geokey name switch of geosearch
Files to link are 34,212 and 5,020 records
Number of pairs that will be compared: 131,999


Matched 0 records; 45.37% still eligible to match


## Pass 3: Block on house number and street name Soundex

In [43]:
pvs_cascade.run_matching_pass(
    pass_name="house number and street name Soundex",
    blocking_columns=["street_number", "street_name_soundex"],
)

Running house number and street name Soundex of geosearch
Files to link are 34,212 and 5,020 records
Number of pairs that will be compared: 214,801


Matched 665 records; 40.82% still eligible to match


## Pass 4: Block on house number and street name Soundex, switching first and last names

In [44]:
pvs_cascade.run_matching_pass(
    pass_name="house number and street name Soundex name switch",
    blocking_columns=["street_number", "street_name_soundex"],
    input_data_transformation=switch_first_and_last_names,
)

Running house number and street name Soundex name switch of geosearch
Files to link are 34,212 and 4,517 records
Number of pairs that will be compared: 170,828


Matched 0 records; 40.82% still eligible to match


## Pass 5: Block on some name and DOB information

In [45]:
pvs_cascade.run_matching_pass(
    pass_name="some name and DOB information",
    blocking_columns=["first_name_2", "last_name_2", "year_of_birth"],
)

Running some name and DOB information of geosearch
Files to link are 34,212 and 4,517 records
Number of pairs that will be compared: 2,410


Matched 2230 records; 30.94% still eligible to match


## Post-process and confirm PIKs

In [46]:
pvs_cascade.confirm_piks()

73 input records linked to deceased people, dropping links
3 input records linked to multiple PIKs, dropping links


,record_id_raw_input_file,pik,record_id_census_2030,record_id_reference_file,module_name,pass_name,match_probability
0,census_2030_0,1142,census_2030_preprocessed_0,geobase_reference_file_12680,geosearch,geokey,1.000000
1,census_2030_10,5405,census_2030_preprocessed_10,geobase_reference_file_15760,geosearch,some name and DOB information,0.999992
2,census_2030_1000,10580,census_2030_preprocessed_1000,geobase_reference_file_8195,geosearch,geokey,1.000000
3,census_2030_10001,3239,census_2030_preprocessed_10001,geobase_reference_file_10521,geosearch,geokey,1.000000
4,census_2030_10002,3238,census_2030_preprocessed_10002,geobase_reference_file_956,geosearch,geokey,1.000000
...,...,...,...,...,...,...,...
7558,census_2030_9987,19,census_2030_preprocessed_9987,geobase_reference_file_12572,geosearch,geokey,1.000000
7559,census_2030_999,5052,census_2030_preprocessed_999,geobase_reference_file_25931,geosearch,house number and street name Soundex,1.000000
7560,census_2030_9992,8072,census_2030_preprocessed_9992,geobase_reference_file_32953,geosearch,some name and DOB information,1.000000
7561,census_2030_9993,1655,census_2030_preprocessed_9993,geobase_reference_file_22697,geosearch,some name and DOB information,1.000000


In [47]:
pvs_cascade.confirmed_piks.groupby(["module_name", "pass_name"]).size().sort_values(ascending=False)

module_name  pass_name                           
geosearch    geokey                                  5983
             some name and DOB information           1083
             house number and street name Soundex     497
dtype: int64

# ZIP3 Adjacency Search

Layne, Wagner, and Rothhaas (p. 6) refers to ZIP3 adjacency as a module and even provides
a list of two passes within it in Appendix A.

Alexander et al. (p. 6) refers to it the same way, as a module.

However, Wagner and Layne (p. 14) says:

> The GeoSearch module also incorporates the adjacency of neighboring areas with
different zip3 values

which implies this logic is a pass or passes *within* GeoSearch (and due to how cascading works,
this is not just an academic distinction).

There is no mention at all of ZIP3 adjacency in the Brown et al. paper from 2023.

**TODO**: Determine whether this still exists, how it works if so, and implement it.
Doesn't really make sense with the sample data since there is only one ZIP.

# Movers

The Movers module is not mentioned in Brown et al, nor in Wagner and Layne.

In Layne, Wagner, and Rothhaas (p. 7), published in 2014, it is described as "Prototyped only – not implemented in PVS":

> The Movers module is appropriate for input files that combine individuals together into
> households.
> The module seeks multiple members of an input household in one address that
> may have moved together to another address.
> To be eligible for this module the household size must be greater than one.
> This module is being tested and was not used in the analysis for this paper.

Alexander et al. (p. 6) a year later makes no note of it being experimental:

> To be eligible for the Movers Module, no member of the household can have a
> PIK and the household must consist of more than one member.
> This module considers persons living at the same address as a unit and
> searches for matching units living together in the reference file (without regard for address).

Essentially the same text is included in Massey et al., published in 2018.

Though it doesn't provide much information, [this document](https://gunnisonconsulting.com/docs/pdf/Record%20Linkage%20Slicksheet_FINAL.pdf)
([archive](https://web.archive.org/web/20230705010237/https://gunnisonconsulting.com/docs/pdf/Record%20Linkage%20Slicksheet_FINAL.pdf))
from the Gunnison consulting group states that they worked with Census and
"pioneered the transition from a record-by-record matching approach to an
approach that better uses groups of records, such as full households, to make more
and better links."
Specifically, they claim that 16% of previously unlinked records were linked using
the household-level approach.

**TODO**: Haven't figured out if this is still in use, nor how it works.
This module seems of particular interest, because it sounds like it is quite different
from other modules.

# NameSearch

Brown et al., p. 29:

> ... the NameSearch module, which uses only name and DOB fields, comparing all combinations of
alternate names and dates of birth.

Wagner and Layne, p. 15:

> The NameSearch module searches the reference files for records failing the
Verification and GeoSearch Modules. Only name and date of birth data are used in this
search process. NameSearch consists of multiple passes against the Numident Name
Reference file, which contains all possible combinations of alternate names and
alternate dates of birth for each SSN in the Census Numident file, and includes data for
ITINs.
> 
> The typical NameSearch blocking strategy starts with a strict first pass, blocking
records by exact date of birth and parts of names. Successive passes block on parts of
the name and date of birth fields to allow for some name and date of birth variation.
The typical match variables are first, middle and last names, generational suffix, date of
birth, and gender.

As in GeoSearch, the only full listing I could find of passes with blocking and matching variables
was in Layne, Wagner, and Rothhaas.
This is probably somewhat out of date and is not what would be used on a CUF, but I have
copied it exactly here.

In [48]:
pvs_cascade.start_module(
    name="namesearch",
    reference_file=name_dob_reference_file,
    reference_file_name="name_dob_reference_file",
    cut_columns=["first_initial_cut", "last_initial_cut"],
    matching_columns=[
        "first_name_15",
        "last_name_12",
        "middle_initial",
        "day_of_birth",
        "month_of_birth",
        "year_of_birth",
    ],
)

## Pass 1: Block on DOB and NYSIIS of name

In [49]:
pvs_cascade.run_matching_pass(
    pass_name="DOB and NYSIIS of name",
    blocking_columns=["day_of_birth", "month_of_birth", "year_of_birth", "first_name_nysiis", "last_name_nysiis"],
)

Running DOB and NYSIIS of name of namesearch
Files to link are 19,532 and 3,495 records
Number of pairs that will be compared: 1,771


Matched 1771 records; 15.58% still eligible to match


## Pass 2: Block on DOB and initials

In [50]:
pvs_cascade.run_matching_pass(
    pass_name="DOB and initials",
    blocking_columns=["day_of_birth", "month_of_birth", "year_of_birth", "first_name_1", "last_name_1"],
)

Running DOB and initials of namesearch
Files to link are 19,532 and 1,724 records
Number of pairs that will be compared: 346


Matched 329 records; 12.62% still eligible to match


## Pass 3: Block on year of birth and first two characters of name

In [51]:
pvs_cascade.run_matching_pass(
    pass_name="year of birth and first two characters of name",
    blocking_columns=["year_of_birth", "first_name_2", "last_name_2"],
)

Running year of birth and first two characters of name of namesearch
Files to link are 19,532 and 1,396 records
Number of pairs that will be compared: 166


Matched 93 records; 11.78% still eligible to match


## Pass 4: Block on birthday and first two characters of name

In [52]:
pvs_cascade.run_matching_pass(
    pass_name="birthday and first two characters of name",
    blocking_columns=["day_of_birth", "month_of_birth", "first_name_2", "last_name_2"],
)

Running birthday and first two characters of name of namesearch
Files to link are 19,532 and 1,303 records
Number of pairs that will be compared: 156


Matched 129 records; 10.61% still eligible to match


## Post-process and confirm PIKs

In [53]:
pvs_cascade.confirm_piks()

74 input records linked to deceased people, dropping links
0 input records linked to multiple PIKs, dropping links


,record_id_raw_input_file,pik,record_id_census_2030,record_id_reference_file,module_name,pass_name,match_probability
0,census_2030_0,1142,census_2030_preprocessed_0,geobase_reference_file_12680,geosearch,geokey,1.000000
1,census_2030_10,5405,census_2030_preprocessed_10,geobase_reference_file_15760,geosearch,some name and DOB information,0.999992
2,census_2030_1000,10580,census_2030_preprocessed_1000,geobase_reference_file_8195,geosearch,geokey,1.000000
3,census_2030_10001,3239,census_2030_preprocessed_10001,geobase_reference_file_10521,geosearch,geokey,1.000000
4,census_2030_10002,3238,census_2030_preprocessed_10002,geobase_reference_file_956,geosearch,geokey,1.000000
...,...,...,...,...,...,...,...
9803,census_2030_9989,16052,census_2030_preprocessed_9989,name_dob_reference_file_17756,namesearch,DOB and NYSIIS of name,1.000000
9804,census_2030_9991,8517,census_2030_preprocessed_9991,name_dob_reference_file_3791,namesearch,DOB and initials,0.998528
9805,census_2030_9997,13395,census_2030_preprocessed_9997,name_dob_reference_file_10374,namesearch,DOB and NYSIIS of name,1.000000
9806,census_2030_9998,14041,census_2030_preprocessed_9998,name_dob_reference_file_4272,namesearch,DOB and NYSIIS of name,1.000000


In [54]:
pvs_cascade.confirmed_piks.groupby(["module_name", "pass_name"]).size().sort_values(ascending=False)

module_name  pass_name                                     
geosearch    geokey                                            5983
namesearch   DOB and NYSIIS of name                            1708
geosearch    some name and DOB information                     1083
             house number and street name Soundex               497
namesearch   DOB and initials                                   320
             birthday and first two characters of name          129
             year of birth and first two characters of name      88
dtype: int64

# DOBSearch

Brown et al., p. 29:

> ... the DOBSearch module, which blocks on
month and day of birth, then compares name, sex, and DOB.

Wagner and Layne, p. 15:

> The DOBSearch module searches the reference files for the records that fail the
> NameSearch, using name and date of birth data. The module matches against a re-split
> version of the Numident Name Reference file, splitting the data based on month and day
> of birth.
> 
> There are typically four blocking passes in the DOBSearch module. The first pass
> blocks records by first name in the incoming file to last name in the DOB Reference file
> and last name in incoming file to first name in the DOB Reference file. This strategy
> accounts for switching of first and last name in the incoming file.

Again, I have used the exact passes listed for this module in Layne, Wagner, and Rothhaas.
This is somewhat corroborated to be "typical" given that it is consistent with Wagner and Layne
above, but both papers are old.

In [55]:
pvs_cascade.start_module(
    name="dobsearch",
    reference_file=name_dob_reference_file,
    reference_file_name="name_dob_reference_file",
    cut_columns=["day_of_birth", "month_of_birth"],
    matching_columns=[
        "first_name_15",
        "last_name_12",
        "middle_initial",
        "day_of_birth",
        "month_of_birth",
        "year_of_birth",
    ],
)

## Pass 1: Block on initials, switching first and last names

In [56]:
pvs_cascade.run_matching_pass(
    pass_name="initials name switch",
    blocking_columns=["first_name_1", "last_name_1"],
    input_data_transformation=switch_first_and_last_names,
)

Running initials name switch of dobsearch
Files to link are 19,532 and 1,248 records
Number of pairs that will be compared: 133


Matched 0 records; 11.28% still eligible to match


## Pass 2: Block on first three characters of name

In [57]:
pvs_cascade.run_matching_pass(
    pass_name="first three characters of name",
    blocking_columns=["first_name_3", "last_name_3"],
)

Running first three characters of name of dobsearch
Files to link are 19,532 and 1,248 records
Number of pairs that will be compared: 81


Matched 67 records; 10.67% still eligible to match


## Pass 3: Block on reverse Soundex of name

In [58]:
pvs_cascade.run_matching_pass(
    pass_name="reverse Soundex of name",
    blocking_columns=["first_name_reverse_soundex", "last_name_reverse_soundex"],
)

Running reverse Soundex of name of dobsearch
Files to link are 19,532 and 1,181 records
Number of pairs that will be compared: 52


Matched 35 records; 10.36% still eligible to match


## Pass 4: Block on first two characters of first name and year of birth

In [59]:
pvs_cascade.run_matching_pass(
    pass_name="first two characters of first name and year of birth",
    blocking_columns=["first_name_2", "year_of_birth"],
)

Running first two characters of first name and year of birth of dobsearch
Files to link are 19,532 and 1,146 records
Number of pairs that will be compared: 202


Matched 165 records; 8.87% still eligible to match


## Post-process and confirm PIKs

In [60]:
pvs_cascade.confirm_piks()

70 input records linked to deceased people, dropping links
0 input records linked to multiple PIKs, dropping links


,record_id_raw_input_file,pik,record_id_census_2030,record_id_reference_file,module_name,pass_name,match_probability
0,census_2030_0,1142,census_2030_preprocessed_0,geobase_reference_file_12680,geosearch,geokey,1.000000
1,census_2030_10,5405,census_2030_preprocessed_10,geobase_reference_file_15760,geosearch,some name and DOB information,0.999992
2,census_2030_1000,10580,census_2030_preprocessed_1000,geobase_reference_file_8195,geosearch,geokey,1.000000
3,census_2030_10001,3239,census_2030_preprocessed_10001,geobase_reference_file_10521,geosearch,geokey,1.000000
4,census_2030_10002,3238,census_2030_preprocessed_10002,geobase_reference_file_956,geosearch,geokey,1.000000
...,...,...,...,...,...,...,...
10000,census_2030_9800,6895,census_2030_preprocessed_9800,name_dob_reference_file_14495,dobsearch,first two characters of first name and year of...,0.994484
10001,census_2030_9813,12585,census_2030_preprocessed_9813,name_dob_reference_file_95,dobsearch,first two characters of first name and year of...,0.999696
10002,census_2030_9855,14355,census_2030_preprocessed_9855,name_dob_reference_file_13208,dobsearch,reverse Soundex of name,0.999997
10003,census_2030_9966,9999,census_2030_preprocessed_9966,name_dob_reference_file_2733,dobsearch,first two characters of first name and year of...,0.994484


In [61]:
pvs_cascade.confirmed_piks.groupby(["module_name", "pass_name"]).size().sort_values(ascending=False)

module_name  pass_name                                           
geosearch    geokey                                                  5983
namesearch   DOB and NYSIIS of name                                  1708
geosearch    some name and DOB information                           1083
             house number and street name Soundex                     497
namesearch   DOB and initials                                         320
dobsearch    first two characters of first name and year of birth     162
namesearch   birthday and first two characters of name                129
             year of birth and first two characters of name            88
dobsearch    reverse Soundex of name                                   35
dtype: int64

# HHCompSearch

Brown et al., p. 29:

> ... the Household Composition search module, which requires at least one
> person in the household of the unmatched person to have received a PIK.
> The full set of unmatched records with historical name, DOB, sex, and address data from
> households whose members with PIKs were observed in the past is created. The module
> attempts to find a match to this universe based on name and DOB information.

Wagner and Layne, p. 16:

> For persons with a PIK in the eligible household, all of the geokeys from the PVS
> GeoBase are extracted for each of these PIKs. The geokeys are unduplicated and all
> persons are selected from the PVS GeoBase with these geokeys. Next, the program
> removes all household members with a PIK, leaving the unPIKed persons in the
> household. This becomes the reference file to search against. There are typically two
> passes in this module. Records are blocked by MAFID, name, date of birth, and gender.

I don't know of any list of passes in this module.
Blocking on "MAFID, name, date of birth, and gender" seems like a lot for only
two passes, especially given that we are only matching people within each eligible
household.
I'm actually a bit surprised there is any blocking at all, given how restrictive this
module inherently is.
I've split the difference here by creating two passes that are very permissive.

## Create the reference file

In [62]:
# TODO: As of now in pseudopeople, our only indicator in the Census data of household
# is the geokey itself. This can be messed up by noise, so we should switch to using
# a (presumably low-noise) household indicator when we have that.
household_id_approximation = (
    census_2030[["geokey"]].drop_duplicates().reset_index()
        .rename(columns={"index": "household_id"})
        .set_index("geokey").household_id
)
census_2030["household_id"] = census_2030.geokey.map(household_id_approximation)

piks_with_household = (
    census_2030[["household_id", "record_id_raw_input_file"]]
        .merge(pvs_cascade.confirmed_piks, on="record_id_raw_input_file", how="left")
)
someone_piked = piks_with_household[piks_with_household.pik.notnull()].groupby("household_id").pik.nunique() > 0
someone_unpiked = piks_with_household[piks_with_household.pik.isnull()].groupby("household_id").pik.nunique() > 0

In [63]:
eligible_households = someone_piked & someone_unpiked

In [64]:
piks_by_household = piks_with_household[["household_id", "pik"]].dropna(subset="pik").drop_duplicates()
piks_by_household

,household_id,pik
0,0,1142
1,1,3117
2,2,2082
3,3,1646
5,3,5327
...,...,...
11046,11047,17649
11047,11047,17632
11049,11047,17634
11050,11047,18278


In [65]:
geokeys_by_household = (
    piks_by_household
        .merge(geobase_reference_file[["pik", "geokey"]].dropna(subset="geokey"), on="pik")
        .drop(columns=["pik"])
        .drop_duplicates()
)
geokeys_by_household

,household_id,geokey
0,0,147-153 BROWNING AVE ANYTOWN US 00000
1,0,147-153 BROWNING AVE ANYTOWN US 02010
2,2,2115 CANNON DR ANYTOWN US 00000
3,3,146 DELAWARE AV ANYTOWN US 00000
6,7,12828 GWENDOLYN DR ANYTOWN US 00000
...,...,...
15016,11044,228 SCHUSTER RD ANYTOWN US 00000
15017,11044,3914 DOOLITTLE LN # 91 ANYTOWN US 00000
15018,11047,207 HARRISON ST ANYTOWN US 00000
15020,11052,4006 ZARAGOZA ST ANYTOWN US 00000


In [66]:
records_to_search_by_household = geokeys_by_household.merge(geobase_reference_file, on="geokey")
records_to_search_by_household

,household_id,geokey,record_id,ssn,first_name,middle_initial,last_name,street_number,street_name,unit_number,...,first_name_15,last_name_12,first_name_1,last_name_1,first_name_2,last_name_2,first_name_3,last_name_3,street_name_soundex,zip3
0,0,147-153 BROWNING AVE ANYTOWN US 00000,geobase_reference_file_12680,332-43-4706,John,E,Mckeever,147-153,BROWNING AVE,None,...,John,Mckeever,J,M,Jo,Mc,Joh,Mck,B655,000
1,0,147-153 BROWNING AVE ANYTOWN US 02010,geobase_reference_file_12681,332-43-4706,John,E,Mckeever,147-153,BROWNING AVE,None,...,John,Mckeever,J,M,Jo,Mc,Joh,Mck,B655,020
2,2,2115 CANNON DR ANYTOWN US 00000,geobase_reference_file_11898,311-66-2007,Gail,K,Durand,2115,CANNON DR,None,...,Gail,Durand,G,D,Ga,Du,Gai,Dur,C553,000
3,3,146 DELAWARE AV ANYTOWN US 00000,geobase_reference_file_3752,101-17-9734,Sarah,L,Williams,146,DELAWARE AV,None,...,Sarah,Williams,S,W,Sa,Wi,Sar,Wil,D461,000
4,3,146 DELAWARE AV ANYTOWN US 00000,geobase_reference_file_25140,665-07-7022,Christian,J,Williams,146,DELAWARE AV,None,...,Christian,Williams,C,W,Ch,Wi,Chr,Wil,D461,000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235655,11047,207 HARRISON ST ANYTOWN US 00000,geobase_reference_file_31419,829-65-4918,Chloe,A,Martinez-Alvarez,207,HARRISON ST,None,...,Chloe,Martinez-Alv,C,M,Ch,Ma,Chl,Mar,H625,000
235656,11047,207 HARRISON ST ANYTOWN US 00000,geobase_reference_file_31762,839-65-3987,Liliana,G,Martinez-Alvarez,207,HARRISON ST,None,...,Liliana,Martinez-Alv,L,M,Li,Ma,Lil,Mar,H625,000
235657,11052,4006 ZARAGOZA ST ANYTOWN US 00000,geobase_reference_file_12583,329-88-8466,Anita,K,Stewart,4006,ZARAGOZA ST,None,...,Anita,Stewart,A,S,An,St,Ani,Ste,Z622,000
235658,11052,4006 ZARAGOZA ST ANYTOWN US 00000,geobase_reference_file_23892,631-80-7414,Steven,R,Stewart,4006,ZARAGOZA ST,None,...,Steven,Stewart,S,S,St,St,Ste,Ste,Z622,000


In [67]:
# Apparently, we exclude from the reference file all *reference file* records with a PIK that
# has already been assigned to an input file row.
# Doing this goes against the normal assumption, which is that reference file records can match
# to multiple input file records.
# This is really surprising to me, but it seems clear from "the program
# removes all household members with a PIK, leaving the unPIKed persons in the
# household. This becomes the reference file to search against." (Wagner and Layne, p. 16)
hhcomp_reference_file = records_to_search_by_household[~records_to_search_by_household.pik.isin(pvs_cascade.confirmed_piks.pik)]
hhcomp_reference_file

,household_id,geokey,record_id,ssn,first_name,middle_initial,last_name,street_number,street_name,unit_number,...,first_name_15,last_name_12,first_name_1,last_name_1,first_name_2,last_name_2,first_name_3,last_name_3,street_name_soundex,zip3
6,3,146 DELAWARE AV ANYTOWN US 00000,geobase_reference_file_31068,820-29-4953,Linda,L,Williams,146,DELAWARE AV,None,...,Linda,Williams,L,W,Li,Wi,Lin,Wil,D461,000
23,12,2545 E GRAND BLVD ANYTOWN US 00000,geobase_reference_file_16691,437-62-1261,Kenneth,A,Deserio,2545,E GRAND BLVD,None,...,Kenneth,Deserio,K,D,Ke,De,Ken,Des,E265,000
24,12,2545 E GRAND BLVD ANYTOWN US 00000,geobase_reference_file_16692,437-62-1261,Kenneth,A,Deserio,2545,E GRAND BLVD,None,...,Kenneth,Deserio,K,D,Ke,De,Ken,Des,E265,000
26,12,2545 E GRAND BLVD ANYTOWN US 00000,geobase_reference_file_33270,878-84-2045,Phylliq,L,Deserio,2545,E GRAND BLVD,None,...,Phylliq,Deserio,P,D,Ph,De,Phy,Des,E265,000
27,12,2545 E GRAND BLVD ANYTOWN US 00000,geobase_reference_file_33271,878-84-2045,Phyllis,L,Deserio,2545,E GRAND BLVD,None,...,Phyllis,Deserio,P,D,Ph,De,Phy,Des,E265,000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235631,11020,626 INVERGARRY BLVD ANYTOWN US 00000,geobase_reference_file_1485,040-53-2575,Liza,A,French,626,INVERGARRY BLVD,None,...,Liza,French,L,F,Li,Fr,Liz,Fre,I516,000
235632,11020,626 INVERGARRY BLVD ANYTOWN US 00000,geobase_reference_file_10749,282-22-8059,William,D,French,626,INVERGARRY BLVD,None,...,William,French,W,F,Wi,Fr,Wil,Fre,I516,000
235633,11020,626 INVERGARRY BLVD ANYTOWN US 00000,geobase_reference_file_17403,457-88-0429,Stephanie,C,French,626,INVERGARRY BLVD,None,...,Stephanie,French,S,F,St,Fr,Ste,Fre,I516,000
235635,11020,626 INVERGARRY BLVD ANYTOWN US 00000,geobase_reference_file_28054,742-90-9955,Jovan,C,Mclane,626,INVERGARRY BLVD,None,...,Jovan,Mclane,J,M,Jo,Mc,Jov,Mcl,I516,000


In [68]:
pvs_cascade.start_module(
    name="hhcompsearch",
    reference_file=hhcomp_reference_file,
    reference_file_name="hhcomp_reference_file",
    cut_columns=["household_id"],
    matching_columns=[
        "first_name_15",
        "last_name_12",
        "middle_initial",
        "day_of_birth",
        "month_of_birth",
        "year_of_birth",
    ],
)

## Pass 1: Block on initials

In [69]:
pvs_cascade.run_matching_pass(
    pass_name="initials",
    blocking_columns=["first_name_1", "last_name_1"],
)

Running initials of hhcompsearch
Files to link are 65,940 and 1,051 records
Number of pairs that will be compared: 549


Matched 36 records; 9.24% still eligible to match


## Pass 2: Block on year of birth

In [70]:
pvs_cascade.run_matching_pass(
    pass_name="year of birth",
    blocking_columns=["year_of_birth"],
)

Running year of birth of hhcompsearch
Files to link are 65,940 and 1,022 records
Number of pairs that will be compared: 1,299


Matched 3 records; 9.21% still eligible to match


## Post-process and confirm PIKs

In [71]:
pvs_cascade.confirm_piks()

37 input records linked to deceased people, dropping links
0 input records linked to multiple PIKs, dropping links


,record_id_raw_input_file,pik,record_id_census_2030,record_id_reference_file,module_name,pass_name,match_probability
0,census_2030_0,1142,census_2030_preprocessed_0,geobase_reference_file_12680,geosearch,geokey,1.000000
1,census_2030_10,5405,census_2030_preprocessed_10,geobase_reference_file_15760,geosearch,some name and DOB information,0.999992
2,census_2030_1000,10580,census_2030_preprocessed_1000,geobase_reference_file_8195,geosearch,geokey,1.000000
3,census_2030_10001,3239,census_2030_preprocessed_10001,geobase_reference_file_10521,geosearch,geokey,1.000000
4,census_2030_10002,3238,census_2030_preprocessed_10002,geobase_reference_file_956,geosearch,geokey,1.000000
...,...,...,...,...,...,...,...
10002,census_2030_9855,14355,census_2030_preprocessed_9855,name_dob_reference_file_13208,dobsearch,reverse Soundex of name,0.999997
10003,census_2030_9966,9999,census_2030_preprocessed_9966,name_dob_reference_file_2733,dobsearch,first two characters of first name and year of...,0.994484
10004,census_2030_9978,684,census_2030_preprocessed_9978,name_dob_reference_file_107,dobsearch,first two characters of first name and year of...,0.994484
10005,census_2030_2298,17695,census_2030_preprocessed_2298,geobase_reference_file_27141,hhcompsearch,year of birth,0.994667


In [72]:
pvs_cascade.confirmed_piks.groupby(["module_name", "pass_name"]).size().sort_values(ascending=False)

module_name   pass_name                                           
geosearch     geokey                                                  5983
namesearch    DOB and NYSIIS of name                                  1708
geosearch     some name and DOB information                           1083
              house number and street name Soundex                     497
namesearch    DOB and initials                                         320
dobsearch     first two characters of first name and year of birth     162
namesearch    birthday and first two characters of name                129
              year of birth and first two characters of name            88
dobsearch     reverse Soundex of name                                   35
hhcompsearch  year of birth                                              2
dtype: int64

In [73]:
pvs_cascade.confirmed_piks

,record_id_raw_input_file,pik,record_id_census_2030,record_id_reference_file,module_name,pass_name,match_probability
0,census_2030_0,1142,census_2030_preprocessed_0,geobase_reference_file_12680,geosearch,geokey,1.000000
1,census_2030_10,5405,census_2030_preprocessed_10,geobase_reference_file_15760,geosearch,some name and DOB information,0.999992
2,census_2030_1000,10580,census_2030_preprocessed_1000,geobase_reference_file_8195,geosearch,geokey,1.000000
3,census_2030_10001,3239,census_2030_preprocessed_10001,geobase_reference_file_10521,geosearch,geokey,1.000000
4,census_2030_10002,3238,census_2030_preprocessed_10002,geobase_reference_file_956,geosearch,geokey,1.000000
...,...,...,...,...,...,...,...
10002,census_2030_9855,14355,census_2030_preprocessed_9855,name_dob_reference_file_13208,dobsearch,reverse Soundex of name,0.999997
10003,census_2030_9966,9999,census_2030_preprocessed_9966,name_dob_reference_file_2733,dobsearch,first two characters of first name and year of...,0.994484
10004,census_2030_9978,684,census_2030_preprocessed_9978,name_dob_reference_file_107,dobsearch,first two characters of first name and year of...,0.994484
10005,census_2030_2298,17695,census_2030_preprocessed_2298,geobase_reference_file_27141,hhcompsearch,year of birth,0.994667


# Resulting PIKs

In [74]:
pik_values = (
    pvs_cascade.confirmed_piks
        .rename(columns={"record_id_raw_input_file": "record_id"})[["record_id", "pik"]]
        .drop_duplicates()
)

In [75]:
census_2030_piked = census_2030_raw_input.copy()
census_2030_piked = census_2030_piked.merge(
    pik_values,
    how="left",
    on="record_id",
    validate="1:1",
)
census_2030_piked

,record_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,relation_to_reference_person,sex,race_ethnicity,pik
0,census_2030_0,John,E,Mcueever,86,06/29/1942,147-153,browning ave,None,Anytown,US,00000,Reference person,Male,Black,1142
1,census_2030_1,Sharon,T,Schmidt,69,10/50/1960,109,stqllion sr,None,Anytown,US,00000,Reference person,Female,White,3117
2,census_2030_2,Gail,K,Durand,77,01/03/1953,2115,cannon dr,None,Anytown,US,00000,Reference person,Female,Multiracial or Other,2082
3,census_2030_3,John,J,Williams,81,11/24/1948,146,delaware av,None,Anytown,US,00000,Reference person,Male,White,1646
4,census_2030_4,Child,L,Wukliamz,81,09/27/1948,146,delaware av,None,Anytown,US,00000,Opp-sex spouse,Female,White,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11048,census_2030_11048,Chloe,A,Maryknez-Alvarez,21,07/12/2008,207,harrison st,None,Anytown,US,00000,Biological child,Female,Latino,17632
11049,census_2030_11049,Zachary,E,Martinez-Alvarez,18,06/29/2011,207,harrison st,None,Anytown,US,00000,Biological child,Male,NaN,NaN
11050,census_2030_11050,Madeline,A,Martinez-Alvarez,16,08/12/2013,207,harrison st,None,Anytown,US,00000,Biological child,Female,Latino,17634
11051,census_2030_11051,Naomi,A,Martinez-Aldarez,1,11/01/2028,207,harrison st,None,Anytown,US,00000,Grandchild,Female,Latino,18278


In [76]:
piked_proportion = census_2030_piked.pik.notnull().mean()
# Compare with 90.28% of input records PIKed in the 2010 CUF,
# as reported in Wagner and Layne, Table 2, p. 18 
print(f'{piked_proportion:.2%} of the input records were PIKed')

90.54% of the input records were PIKed


In [77]:
census_2030_piked.to_parquet('census_2030_piked_sample.parquet')

# PIK accuracy

Here we inspect the accuracy and characteristics of the PIKs assigned,
leveraging our knowledge of ground truth from pseudopeople.

It wouldn't be possible to do exactly this with the real PVS, but
Layne, Wagner, and Rothhaas did something similar by redacting SSN from real records,
sending them through PVS without the SSN, and then using the true SSN
as ground truth.
The health care records they used are probably quite different from a CUF,
but they found a **very** good overall PIK accuracy (see cell below).

In [78]:
# All modules, Medicare database, calculated from Layne, Wagner, and Rothhaas Table 1 (p. 15)
real_life_pvs_accuracy = 1 - (2_585 + 60_709 + 129_480 + 89_094) / (52_406_981 + 5_170_924 + 49_374_794 + 50_327_034)
f'{real_life_pvs_accuracy:.5%}'

'99.82079%'

In [79]:
census_2030_ground_truth = (
    pd.read_parquet('generate_simulated_data/output/census_2030_ground_truth_sample.parquet')
        .set_index('record_id').simulant_id
)

In [80]:
reference_files_ground_truth = pd.concat([
    pd.read_parquet('generate_simulated_data/output/geobase_reference_file_ground_truth_sample.parquet'),
    pd.read_parquet('generate_simulated_data/output/name_dob_reference_file_ground_truth_sample.parquet'),
], ignore_index=True).set_index('record_id').simulant_id
reference_files_ground_truth

record_id
geobase_reference_file_0         0_13602
geobase_reference_file_1         0_16514
geobase_reference_file_2         0_16514
geobase_reference_file_3         0_16514
geobase_reference_file_4         0_13906
                                  ...   
name_dob_reference_file_19527     0_3634
name_dob_reference_file_19528    0_13204
name_dob_reference_file_19529    0_19917
name_dob_reference_file_19530     0_3731
name_dob_reference_file_19531      0_670
Name: simulant_id, Length: 53744, dtype: object

In [81]:
possible_to_pik_proportion = census_2030_ground_truth.isin(reference_files_ground_truth).mean()
print(
    f'{(1 - possible_to_pik_proportion):.2%} of the input records are '
    'impossible to PIK correctly, since they are not in any reference files'
)

3.93% of the input records are impossible to PIK correctly, since they are not in any reference files


In [82]:
print(
    f'Assigned PIKs to {(piked_proportion / possible_to_pik_proportion):.2%} of PIK-able records'
)

Assigned PIKs to 94.24% of PIK-able records


In [83]:
# Multiple Census rows assigned the same PIK, indicating the model thinks they are duplicates in Census
census_2030_piked.pik.value_counts().value_counts()

count
1    9989
2       9
Name: count, dtype: int64

In [84]:
# However, in this version of pseudopeople, there are no actual duplicates in Census
assert not census_2030_ground_truth.duplicated().any()

In [85]:
# Interesting: in pseudopeople, sometimes siblings are assigned the same (common) first name, making them almost identical.
# The only giveaway is their age and DOB.
# Presumably, this tends not to happen in real life.
duplicate_piks = census_2030_piked.pik.value_counts()[census_2030_piked.pik.value_counts() > 1].index
census_2030_piked[census_2030_piked.pik.isin(duplicate_piks)].sort_values('pik')

,record_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,relation_to_reference_person,sex,race_ethnicity,pik
3455,census_2030_3455,Joseph,E,Engle,80,12/02/1949,341,gardner drive,None,Anytown,US,00000,Reference person,Male,White,1735
3456,census_2030_3456,Lorraine,D,Engle,80,10/02/1949,341,gardner drive,None,Anytown,US,00000,Opp-sex spouse,Female,White,1735
10221,census_2030_10221,Ainsley,S,Alsandor,12,12/28/2017,110,30th ave n,None,Anytown,US,00000,Sibling,Female,Black,8571
2016,census_2030_2016,Heidi,E,Alsandor,39,04/16/1990,3232,maple grove ln,None,Anytown,US,00000,Noninstitutionalized GQ pop,Female,Black,8571
7041,census_2030_7041,None,C,Avila,1,01/06/2029,4738,klauber ave,None,Anytown,US,00000,Grandchild,Male,Latino,12781
7032,census_2030_7032,Carlo,C,Avila,21,01/27/2009,4738,klauber ave,None,Anytown,US,00000,Biological child,Male,NaN,12781
1031,census_2030_1031,Steven,R,Maurer,21,02/20/2009,2860,links ct,None,Anytown,US,00000,Reference person,Male,White,12793
3642,census_2030_3642,Steven,R,Maurer,13,04/30/2016,412,key street,None,Anytown,US,00000,Biological child,Male,White,12793
8379,census_2030_8379,Emily,K,Allen,16,07/30/2013,3076,hanna ave n,None,Anytown,US,00000,NaN,Female,White,13252
8378,census_2030_8378,Emily,K,Allen,18,04/14/2011,3076,hanna ave n,None,Anytown,US,00000,Biological child,Female,White,13252


In [86]:
pik_to_simulant_ground_truth = (
    pd.read_parquet('generate_simulated_data/output/pik_to_simulant_ground_truth.parquet')
        .set_index("pik").simulant_id
)
pik_to_simulant_ground_truth

pik
452      0_13602
5866     0_16514
4761     0_13906
4172     0_13442
17384    0_22495
          ...   
91100    0_19917
91101     0_3731
11130     0_9017
91102      0_670
4027     0_12964
Name: simulant_id, Length: 18907, dtype: object

In [87]:
matched_simulant_id = census_2030_piked.set_index("record_id").pik.map(pik_to_simulant_ground_truth)

In [88]:
census_2030_ground_truth[matched_simulant_id.notnull()]

record_id
census_2030_0          0_923
census_2030_1         0_2641
census_2030_2         0_6176
census_2030_3        0_13972
census_2030_5        0_13974
                      ...   
census_2030_11047    0_22740
census_2030_11048    0_22741
census_2030_11050    0_22743
census_2030_11051    0_23271
census_2030_11052    0_16724
Name: simulant_id, Length: 10007, dtype: object

In [89]:
piks_correct_proportion = (matched_simulant_id[matched_simulant_id.notnull()] == census_2030_ground_truth[matched_simulant_id.notnull()]).mean()
print(f'{piks_correct_proportion:.5%} of the PIKs assigned were correct; compare with {real_life_pvs_accuracy:.5%} in real life')

99.88008% of the PIKs assigned were correct; compare with 99.82079% in real life


In [90]:
confirmed_piks_with_ground_truth = pvs_cascade.confirmed_piks.copy()
confirmed_piks_with_ground_truth["correct"] = (
    confirmed_piks_with_ground_truth.record_id_raw_input_file
        .map(census_2030_ground_truth)
    ==
    confirmed_piks_with_ground_truth.record_id_reference_file
        .map(reference_files_ground_truth)
)
confirmed_piks_with_ground_truth.correct.mean()

0.9988008394124113

In [91]:
# Accuracy by module -- note that this shows the opposite pattern (with the sample data)
# relative to the results of Layne et al., who found GeoSearch was much *more* accurate
confirmed_piks_with_ground_truth.groupby(["module_name"]).correct.mean().sort_values()

module_name
geosearch       0.998413
dobsearch       1.000000
hhcompsearch    1.000000
namesearch      1.000000
Name: correct, dtype: float64

In [92]:
# Accuracy by pass -- could be used to tune pass-specific cutoffs, but
# this might not be too informative while we are still using the sample data.
confirmed_piks_with_ground_truth.groupby(["module_name", "pass_name"]).correct.mean().sort_values()

module_name   pass_name                                           
geosearch     geokey                                                  0.997994
dobsearch     first two characters of first name and year of birth    1.000000
              reverse Soundex of name                                 1.000000
geosearch     house number and street name Soundex                    1.000000
              some name and DOB information                           1.000000
hhcompsearch  year of birth                                           1.000000
namesearch    DOB and NYSIIS of name                                  1.000000
              DOB and initials                                        1.000000
              birthday and first two characters of name               1.000000
              year of birth and first two characters of name          1.000000
Name: correct, dtype: float64

### Incorrect PIKs

In [93]:
(
    confirmed_piks_with_ground_truth[~confirmed_piks_with_ground_truth.correct]
        .groupby(["module_name", "pass_name"])
        .size().sort_values()
)

module_name  pass_name
geosearch    geokey       12
dtype: int64

In [94]:
incorrectly_linked_pairs = (
    confirmed_piks_with_ground_truth[~confirmed_piks_with_ground_truth.correct]
        [["record_id_census_2030", "record_id_reference_file"]].drop_duplicates()
)
incorrectly_linked_pairs

,record_id_census_2030,record_id_reference_file
167,census_2030_preprocessed_10221,geobase_reference_file_16646
308,census_2030_preprocessed_10402,geobase_reference_file_34166
890,census_2030_preprocessed_1243,geobase_reference_file_12650
2637,census_2030_preprocessed_3455,geobase_reference_file_3358
2783,census_2030_preprocessed_3642,geobase_reference_file_2894
3687,census_2030_preprocessed_4838,geobase_reference_file_5813
4151,census_2030_preprocessed_5458,geobase_reference_file_11840
4153,census_2030_preprocessed_5460,geobase_reference_file_1062
5313,census_2030_preprocessed_6973,geobase_reference_file_2313
5365,census_2030_preprocessed_7041,geobase_reference_file_28115


In [95]:
census_incorrectly_linked = (
    census_2030
        .set_index("record_id")
        .loc[incorrectly_linked_pairs["record_id_census_2030"], :]
        .reset_index()
)

reference_file_incorrectly_linked = (
    pd.concat([geobase_reference_file, name_dob_reference_file], ignore_index=True)
        .set_index("record_id")
        .loc[incorrectly_linked_pairs["record_id_reference_file"], :]
        .reset_index()
)

comparison_cols = [
    "first_name",
    "middle_initial",
    "last_name",
    "year_of_birth",
    "month_of_birth",
    "day_of_birth",
    "geokey",
]

census_incorrectly_linked[comparison_cols].compare(
    reference_file_incorrectly_linked[comparison_cols],
    keep_shape=True,
    keep_equal=True,
)

first_name           middle_initial               last_name  \
         self     other           self other              self   
0     Ainsley     Heidi              S     S          Alsandor   
1      Sophia    Lazaro              D     D     Jimenez Perez   
2         Jax   Jessica              C     C           Kuhnert   
3      Joseph  Lorraine              E     D             Engle   
4      Steven    Steven              R     R            Maurer   
5     Yaretzi    Justin              E     A           Johnson   
6       Mason    Thomas              A     A            Reeves   
7     Gregory     Jason              J     J  Jimenez Santiago   
8        Ryan     Kaden              D     H            Nguyen   
9        None     Carlo              C     C             Avila   
10    Camilla     Reese              J     H   Martinez-Jaimes   
11      Emily     Emily              K     K             Allen   

                     year_of_birth         month_of_birth       day_of_birth  \
               other          self   other           self other         self   
0           Alsandor        2017.0     NaN           12.0   NaN         28.0   
1      Jimenez Perez        2012.0     NaN            1.0   NaN          7.0   
2            Kuhnert           NaN  2005.0            NaN   4.0          NaN   
3              Engle        1949.0  1949.0           12.0  10.0          2.0   
4             Maurer        2016.0  2009.0            4.0   2.0         30.0   
5            Johgson        2022.0  2022.0            6.0   6.0         17.0   
6             Reeves           NaN  1981.0            NaN   3.0          NaN   
7   Jimenez Santiago        1977.0  2002.0            7.0   7.0          2.0   
8             Nguyen           NaN  2019.0            NaN   3.0          NaN   
9              Avila        2029.0  2009.0            1.0   1.0          6.0   
10   Martinez-Jaimes        2025.0  2025.0            9.0   9.0         22.0   
11             Allen        2013.0  2011.0            7.0   4.0         30.0   

                                         geokey  \
   other                                   self   
0    NaN        110 30TH AVE N ANYTOWN US 00000   
1    NaN     201 BOEHLKE DRIVE ANYTOWN US 00000   
2   24.0       1100 2ND STREET ANYTOWN US 00000   
3    2.0     341 GARDNER DRIVE ANYTOWN US 00000   
4   20.0        412 KEY STREET ANYTOWN US 00000   
5   17.0        53 EAST AVENUE ANYTOWN US 00000   
6    2.0        20 GIST AVENUE ANYTOWN US 00000   
7    4.0  15 CASTLE DR FLAT 28 ANYTOWN US 00000   
8   21.0  5104 SW CIRRUS DR UG ANYTOWN US 00000   
9   27.0      4738 KLAUBER AVE ANYTOWN US 00000   
10  22.0   44028 8TH AVE SOUTH ANYTOWN US 00000   
11  14.0      3076 HANNA AVE N ANYTOWN US 00000   

                                           
                                    other  
0         110 30TH AVE N ANYTOWN US 00000  
1      201 BOEHLKE DRIVE ANYTOWN US 00000  
2        1100 2ND STREET ANYTOWN US 00000  
3      341 GARDNER DRIVE ANYTOWN US 00000  
4         412 KEY STREET ANYTOWN US 00000  
5         53 EAST AVENUE ANYTOWN US 00000  
6         20 GIST AVENUE ANYTOWN US 00000  
7   15 CASTLE DR FLAT 28 ANYTOWN US 00000  
8   5104 SW CIRRUS DR UG ANYTOWN US 00000  
9       4738 KLAUBER AVE ANYTOWN US 00000  
10   44028 8TH AVE SOUTH ANYTOWN US 00000  
11      3076 HANNA AVE N ANYTOWN US 00000

In [96]:
census_2030_piked.to_parquet('census_2030_piked_sample.parquet')

In [97]:
# Convert this notebook to a Python script
! ./convert_notebook.sh pvs_like_case_study_sample_data

[NbConvertApp] Converting notebook pvs_like_case_study_sample_data.ipynb to python
